In [1]:
#step one, get the top 100
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sounddevice as sd
from scipy.io.wavfile import write
import os
from pydub import AudioSegment
import ffmpeg
from ShazamAPI import Shazam
import json

In [2]:
def doShazam():
    mp3_file_content_to_recognize = open('song.mp3', 'rb').read()

    shazam = Shazam(mp3_file_content_to_recognize)
    recognize_generator = shazam.recognizeSong()

    return (next(recognize_generator)) 

In [3]:
def get_shazam_title():
    #getAudio()
    doShazam()

    if is_recognizable():
        return get_song_info()
    else:
        return "empty"

In [4]:
def soundex(name):


    """
    The Soundex algorithm assigns a 1-letter + 3-digit code to strings,
    the intention being that strings pronounced the same but spelled
    differently have identical encodings; words pronounced similarly
    should have similar encodings.
    """

    soundexcoding = [' ', ' ', ' ', ' ']
    soundexcodingindex = 1

    #           ABCDEFGHIJKLMNOPQRSTUVWXYZ
    mappings = "01230120022455012623010202"

    soundexcoding[0] = name[0].upper()

    for i in range(1, len(name)):

         c = ord(name[i].upper()) - 65

         if c >= 0 and c <= 25:

             if mappings[c] != '0':

                 if mappings[c] != soundexcoding[soundexcodingindex-1]:

                     soundexcoding[soundexcodingindex] = mappings[c]
                     soundexcodingindex += 1

                 if soundexcodingindex > 3:

                     break

    if soundexcodingindex <= 3:
        while(soundexcodingindex <= 3):
            soundexcoding[soundexcodingindex] = '0'
            soundexcodingindex += 1

    return ''.join(soundexcoding)

In [5]:
def is_recognizable():
    return len(shazam_info[1]["matches"])>0

def get_song_info():
    return shazam_info[1]["track"]["title"], shazam_info[1]["track"]["subtitle"]

In [6]:
URL = 'https://www.billboard.com/charts/hot-100/'
Top100Website = requests.get(URL)
result = BeautifulSoup(Top100Website.content, 'html.parser')

In [7]:
result.find("span", id="count_texttitle-of-a-story")

In [8]:
line_number = 0
place_number = 0
Song = "";
Artists = "";
Top_100 = pd.DataFrame(columns=['place','song','artists', 'song_soundex'])
print(str(Top_100))

for lines in result.find_all("li", {"o-chart-results-list__item"}):
    line_number += 1

 
    if((line_number-4)%14==0):
        place_number += 1
        internal_line = 1
        for current_line in lines.get_text().splitlines():

            if(current_line.strip()!=""):
                if(internal_line==1):
                    Song = current_line.strip()
                else:
                    Artists = current_line.strip()

                    Top_100.loc[place_number -1] = [place_number, Song, Artists, soundex(str(Song))]
 
                internal_line += 1

Empty DataFrame
Columns: [place, song, artists, song_soundex]
Index: []


In [9]:
#temporary
Top_100

,place,song,artists,song_soundex
0,1,As It Was,Harry Styles,A232
1,2,First Class,Jack Harlow,F623
2,3,Heat Waves,Glass Animals,H312
3,4,Big Energy,Latto,B256
4,5,Enemy,Imagine Dragons X JID,E500
...,...,...,...,...
95,96,P Power,Gunna Featuring Drake,P160
96,97,Money So Big,Yeat,M521
97,98,Blick Blick!,Coi Leray & Nicki Minaj,B421
98,99,Fall In Love,Bailey Zimmerman,F454


In [18]:
recommend_me_something_for_this = input("Please enter a song to base a recommendation on ")

Please enter a song to base a recommendation on love


In [19]:
if(recommend_me_something_for_this=="shazaam"):
    
    fs = 44100  # this is the frequency sampling; also: 4999, 64000
    seconds = 5  # Duration of recording

    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    print("Starting: Speak now!")
    sd.wait()  # Wait until recording is finished
    print("finished")
    write('song.wav', fs, myrecording)
    
    sound = AudioSegment.from_wav("song.wav")
    
    # comment this for testing 
    # sound.export('song.mp3', format='mp3')
    
    mp3_file_content_to_recognize = open('song.mp3', 'rb').read()

    shazam = Shazam(mp3_file_content_to_recognize)
    recognize_generator = shazam.recognizeSong()
    
    shazam_info = next(recognize_generator)
                   
    if(len(shazam_info[1]["matches"])>0):
                   
       print("that was : " + shazam_info[1]["track"]["title"] + " from :" + shazam_info[1]["track"]["subtitle"])
       recommend_me_something_for_this = shazam_info[1]["track"]["title"]
                   
    
Number_Of_Songs_Found = len(Top_100[Top_100['song_soundex'] == soundex(recommend_me_something_for_this.strip())])
if(Number_Of_Songs_Found>0):
    #find a new song
    print("Let me recommend you:")
    recommendation_dataframe = Top_100.sample(n=1)
    print(str(recommendation_dataframe ))
else:
    print("“Unfortunately, the song is not in the hot list!")    

“Unfortunately, the song is not in the hot list!


SyntaxError: unexpected EOF while parsing (1224295644.py, line 1)